# Preparando o ambiente (source)

Para utilizar corretamente o cluster, a primeira coisa que deve ser feita **dentro** da chococino é atualizar as variáveis de ambiente:

```sh
$ source /home/prof/hadoop/bin/chococino_env

# Testando para ver se foi tudo configurado
$ echo JAVA_HOME
/usr/lib/jvm/java-8-openjdk-amd64/jre
```

# Redirecionando as portas (ssh port fowarding)

Após instanciada as variáveis, se faz necessário realizar o redirecionamento das portas das interfaces gráficas (spark = 9090, hadoop = 9870 e notebook). Para isso rode o seguinte comando **fora** da chococino:

```sh
$ ssh -L <porta_local>:<ip_da_maquina>:<porta> <usuario>@chococino.naquadah.com.br -p13508

# Exemplo de redirecionamento das portas
$ ssh -L 9090:cm1:9090 -L 9870:chococino:9870 -L 8888:gpu1:8888 a190030879@chococino.naquadah.com.br -p13508
```

Obs.: Não é necessário fazer o redirecionamento das portas do zookeeper ou kafka (a princípio)

Obs2.: Caso queira, é possível fazer o redirecionamento da porta 9092 (kafka-server) de qualquer máquina (com exceção da chococino) para se comunicar diretamente com a fila do Kafka pelo computador local.

Obs3.: No caso de redirecionamento do Kafka é necessário ter o binário do kafka-console-{consumer/producer} no computador na versão 2.13-3.3.1. Com isso basta passar a flag **bootstrap-server** como localhost:<porta_aberta>

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("spark://cm1:7077").appName("teste kafka").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/07 19:14:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
lines = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "cm4:9092") \
    .option("subscribe", "teste") \
    .option('includeTimestamp', 'true') \
    .load()

In [4]:
from pyspark.sql.functions import explode, split, col

In [5]:
words = lines.select(
    explode(split(col("value"), "\s+")).alias("word"),
    lines.timestamp
)

## Kafka Producer (terminal) - Consumer (notebook)
- Em outro terminal, abra um producer do kafka com o seguinte comando:
```sh
kafka-console-producer.sh --bootstrap-server cm2:9092 --topic teste
```

- A partir desse ponto é só digitar no terminal do consumidor que os valores começaram a aparecer nas cells abaixo
- Obs.: Aqui já da pra ver que não importa para qual máquina que o programa está enviando as mensagens no kafka, porque tudo está sendo compartilhado


In [6]:
words.writeStream \
    .format('console') \
    .option('truncate', 'false') \
    .start()

23/02/05 19:51:23 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-e7f0bdf1-cb15-4d13-addc-36a07bbf8b7e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/02/05 19:51:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+---------+
|word|timestamp|
+----+---------+
+----+---------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----------------------+
|word    |timestamp              |
+--------+-----------------------+
|testando|2023-02-05 19:51:29.465|
|1       |2023-02-05 19:51:29.465|
|2       |2023-02-05 19:51:29.465|
+--------+-----------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----------------------+
|word |timestamp              |
+-----+-----------------------+
|ola  |2023-02-05 19:51:32.174|
|mundo|2023-02-05 19:51:32.174|
+-----+-----------------------+

-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----------------------+
|word|timestamp              |
+----+-----------------------+
|tudo|2023-02-05 19:51:38.594|
|bem |2023-02-05 19:51:38.594|
|?   |2023-02-05 19:51:38.594|
+----+-----------------------+



In [7]:
spark.stop()